In [320]:
import pandas as pd
import numpy as np

In [260]:
import warnings
warnings.filterwarnings('ignore')

### 1. Загрузка и предобработка данных

- 📥 **Загрузите набор данных** Bank Customer Churn Prediction и обработайте пропуски.

In [345]:
def read_csv():
    return pd.read_csv("Bank Customer Churn Prediction.csv")

data = read_csv()
data.dropna(inplace=True)

# Есть также способ:
# data = data.dropna().copy(deep=True)

- 🔄 **Разделите данные на признаки (X) и целевую переменную (y).**

In [346]:
X = data.drop(['churn'], axis=1)
y = data['churn']

- 📈 **Выведите корреляционную сортированную таблицу** признаков и таргета для определения самых важных признаков, которые влияют на целевую переменную (либо в виде сортированного графика).

PS: В таблице находятся качественные переменные, чтобы сравнить корреляцию я (Каримов Шохин) хочу сделать качественные переменные
фиктивными. Но обычно при обучении модели (в регрессии в частности) одно значение качественных переменных удаляется чтобы не было мультиколлиниальности. Я не знаю как вы будете считать правильным для сравнение корреляции независимых переменных с таргетом (оставление всех значений качественных переменных или удаление одного с каждого), поэтому я создам оба варианта. 

Первый вариант со всеми значениями

In [344]:
data_encoded = pd.get_dummies(data, dtype=int)

In [264]:
data_encoded.corr()['churn'].sort_values(ascending=False)

churn               1.000000
age                 0.285323
country_Germany     0.173488
balance             0.118533
gender_Female       0.106512
estimated_salary    0.012097
customer_id        -0.006248
credit_card        -0.007138
tenure             -0.014001
credit_score       -0.027094
products_number    -0.047820
country_Spain      -0.052667
country_France     -0.104955
gender_Male        -0.106512
active_member      -0.156128
Name: churn, dtype: float64

Второй вариант с удаленными первыми значениями с каждой колонки

In [265]:
data_encoded = pd.get_dummies(data, drop_first=True, dtype=int)

In [266]:
data_encoded.corr()['churn'].sort_values(ascending=False)

churn               1.000000
age                 0.285323
country_Germany     0.173488
balance             0.118533
estimated_salary    0.012097
customer_id        -0.006248
credit_card        -0.007138
tenure             -0.014001
credit_score       -0.027094
products_number    -0.047820
country_Spain      -0.052667
gender_Male        -0.106512
active_member      -0.156128
Name: churn, dtype: float64

- 📊 **Выведите распределение целевой переменной в процентном соотношении.**


In [267]:
y.value_counts(normalize=True)

churn
0    0.7963
1    0.2037
Name: proportion, dtype: float64

### 2. Удерживающая проверка (Hold-Out CV)

In [268]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

- ✂️ **Разделите данные на обучающую и тестовую выборки** в соотношении 80/20.

In [347]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 11)

- 🧠 **Обучите модель** CatBoostClassifier с использованием обучающей выборки.

In [348]:
cat_features = ['country', 'gender']

train_pool = Pool(data = X_train, label = y_train, cat_features = cat_features)

test_pool = Pool(data = X_test, label = y_test, cat_features = cat_features)

In [349]:
model = CatBoostClassifier(random_state=11, depth=2, eval_metric='AUC', verbose=0)
model.fit(train_pool, eval_set = test_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

- 📊 **Оцените её производительность** на тестовой выборке, используя метрику AUC.

In [329]:
scores_table = pd.DataFrame(columns=['name', 'AUC_mean', 'AUC_std', 'Accuracy_mean', 'Accuracy_std'])

Оценка AUC (способ №1)

In [272]:
y_pred_proba_test = model.predict_proba(test_pool)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_proba_test)

In [273]:
auc_test

0.8643278816052797

Оценка AUC (способ №2)

In [308]:
metrics_auc = model.eval_metrics(test_pool, metrics=['AUC'], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [309]:
auc_test = metrics_auc['AUC'][-1]
print(f'AUC: {auc_test}')

AUC: 0.8643278816052798


Оценка Accuracy

In [310]:
metrics_accuracy = model.eval_metrics(test_pool, metrics=['Accuracy'], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [311]:
accuracy = metrics_accuracy['Accuracy'][-1]
print(f'Accuracy: {accuracy}')

Accuracy: 0.8595


In [322]:
np.std(metrics_auc['AUC'])

0.01446745021507512

In [330]:
scores_table.loc[len(scores_table)] = ['Hold-Out CV', np.mean(metrics_auc['AUC']), np.std(metrics_auc['AUC']), 
                                                    np.mean(metrics_accuracy['Accuracy']), np.std(metrics_accuracy['Accuracy'])]

In [325]:
scores_table

,name,AUC_mean,AUC_std,Accuracy_mean,Accuracy_std
0,Hold-Out CV,0.855063,0.014467,0.857029,0.00764


### 3. k-блочная перекрестная проверка (k-Fold CV)

- 🔁 **Реализуйте k-Fold перекрестную проверку** с использованием 5 блоков.

In [278]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [279]:
kf = KFold(n_splits=5, shuffle=True, random_state=11)

In [280]:
kf_scores_accuracy = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int), y, cv=kf, scoring='accuracy')

In [281]:
kf_scores_auc = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int), y, cv=kf, scoring='roc_auc')

- 📊 **Вычислите среднее значение AUC** для всех блоков.

In [331]:
scores_table.loc[len(scores_table)] = ['k-Fold CV', kf_scores_auc.mean(), kf_scores_auc.std(), 
                                            kf_scores_accuracy.mean(),  kf_scores_accuracy.std()]

In [332]:
scores_table

,name,AUC_mean,AUC_std,Accuracy_mean,Accuracy_std
0,Hold-Out CV,0.855063,0.014467,0.857029,0.007640
1,k-Fold CV,0.867098,0.013899,0.862900,0.003904


### 4. Стратифицированный k-Fold CV

- ⚖️ **Выполните стратифицированную k-Fold перекрестную проверку**.

In [285]:
from sklearn.model_selection import StratifiedKFold

In [286]:
skf = StratifiedKFold(n_splits=5)

In [287]:
skf_scores_accuracy = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int), y, cv=skf, scoring='accuracy')

In [288]:
skf_scores_auc = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int), y, cv=skf, scoring='roc_auc')

- 🔍 **Сравните результаты** со стандартной k-Fold проверкой.

In [333]:
scores_table.loc[len(scores_table)] = ['Stratified k-Fold CV', skf_scores_auc.mean(), skf_scores_auc.std(), 
                                            skf_scores_accuracy.mean(), skf_scores_accuracy.std()]

In [334]:
scores_table

,name,AUC_mean,AUC_std,Accuracy_mean,Accuracy_std
0,Hold-Out CV,0.855063,0.014467,0.857029,0.007640
1,k-Fold CV,0.867098,0.013899,0.862900,0.003904
2,Stratified k-Fold CV,0.867336,0.002081,0.864400,0.007053


- 📝 **Объясните, в чем преимущество стратифицированного подхода.**

Преимущество стратифицированного подхода в сохранении пропорций в каждом блоке (подгруппе), что обеспечивает репрезентативность данных. Это дает возможность получить более корректные значения

### 5. Перекрестная проверка с исключением одного (LOOCV)

- 🔄 **Реализуйте метод LOOCV**.

In [296]:
from sklearn.model_selection import LeaveOneOut

In [297]:
loo = LeaveOneOut()

In [303]:
loo_scores_auc = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int).head(400), y.head(400), cv=loo, scoring='roc_auc', n_jobs=-1)

In [304]:
loo_scores_accuracy = cross_val_score(model, pd.get_dummies(X, drop_first=True, dtype=int).head(400), y.head(400), cv=loo, scoring='accuracy', n_jobs=-1)

- 📊 **Сравните полученные результаты** с предыдущими методами. Если алгоритм работает долго, попробуйте сократить количество подаваемых строк.

In [335]:
scores_table.loc[len(scores_table)] = ['LOO CV', loo_scores_auc.mean(), loo_scores_auc.std(), 
                                            loo_scores_accuracy.mean(), loo_scores_accuracy.std()]

In [336]:
scores_table

,name,AUC_mean,AUC_std,Accuracy_mean,Accuracy_std
0,Hold-Out CV,0.855063,0.014467,0.857029,0.007640
1,k-Fold CV,0.867098,0.013899,0.862900,0.003904
2,Stratified k-Fold CV,0.867336,0.002081,0.864400,0.007053
3,LOO CV,NaN,NaN,0.855000,0.352101


### 6. Подготовка сравнительной таблицы

- 📊 **Подготовьте сравнительную таблицу для 4 методов валидации точности модели** (type of CV, std, mean).

In [337]:
scores_table

,name,AUC_mean,AUC_std,Accuracy_mean,Accuracy_std
0,Hold-Out CV,0.855063,0.014467,0.857029,0.007640
1,k-Fold CV,0.867098,0.013899,0.862900,0.003904
2,Stratified k-Fold CV,0.867336,0.002081,0.864400,0.007053
3,LOO CV,NaN,NaN,0.855000,0.352101


### 7. Задание на доп балл

- 🏆 **Выведите таблицу с метриками** для порогов отсечения 0.05, 0.1, 0.15, 0.25, 0.3 для положительного класса 
(Только для Hold-Out CV).

In [350]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [411]:
y_probs = model.predict_proba(X_test)[:, 1]
thresholds = [0.05, 0.1, 0.15, 0.25, 0.3]
# Можно было использовать:
# thresholds = [x for x in np.arange(0.05, 0.31, 0.05)]
# Но также появиться значение 0.2, а так как его нет в условии этот способ я не использовал

In [406]:
metrics_info = pd.DataFrame(columns=['Threshold', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
metrics_info['Threshold'] = thresholds

In [407]:
def get_matrics(y_test, y_probs, threshold):
    y_pred = (threshold <= y_probs).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return [accuracy, precision, recall, f1]

In [408]:
metrics_array = [[] for x in range(4)]
for threshold in thresholds:
    matrics = get_matrics(y_test, y_probs, threshold)
    metrics_array[0].append(matrics[0])
    metrics_array[1].append(matrics[1])
    metrics_array[2].append(matrics[2])
    metrics_array[3].append(matrics[3])

In [409]:
for column in range(4):
    metrics_info[metrics_info.columns[column+1]] = metrics_array[column]

In [410]:
metrics_info

,Threshold,Accuracy,Precision,Recall,F1 Score
0,0.05,0.4230,0.252941,0.972362,0.401452
1,0.10,0.6225,0.335787,0.917085,0.491582
2,0.15,0.7275,0.408922,0.829146,0.547718
3,0.25,0.8095,0.515206,0.723618,0.601881
4,0.30,0.8350,0.574236,0.660804,0.614486
